<a href="https://colab.research.google.com/github/tuantran221/Project_Machine_Learning/blob/main/Loading_in_your_own_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>